In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data_binary = pd.read_csv(r'.\TrafficLabelling\data_binary.csv')
data_multiclass = pd.read_csv(r'.\TrafficLabelling\data_multi_class.csv')

In [5]:
data_binary.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [6]:
summary = data_binary.describe().T[['min','max','mean']]
# Hiển thị toàn bộ bảng (không bị cắt)
pd.set_option('display.max_rows', None)  # hiển thị tất cả dòng
pd.set_option('display.max_columns', None)  # hiển thị tất cả cột

# Hiển thị số ở dạng thường, làm tròn 2 chữ số thập phân
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

print(summary)


                                           min              max          mean
Destination Port                          0.00        65,535.00      8,077.64
Flow Duration                           -13.00   119,999,998.00 14,792,023.78
Total Fwd Packets                         1.00       219,759.00          9.36
Total Backward Packets                    0.00       291,922.00         10.40
Total Length of Fwd Packets               0.00    12,900,000.00        548.32
Total Length of Bwd Packets               0.00   655,453,030.00     16,172.19
Fwd Packet Length Max                     0.00        24,820.00        207.72
Fwd Packet Length Min                     0.00         2,325.00         18.73
Fwd Packet Length Mean                    0.00         5,940.86         58.24
Fwd Packet Length Std                     0.00         7,125.60         68.95
Bwd Packet Length Max                     0.00        19,530.00        871.44
Bwd Packet Length Min                     0.00         2,896.00 

In [7]:
print("Tổng số giá trị thiếu:", data_binary.isnull().sum().sum())
print("Tổng số giá trị thiếu:", data_multiclass.isnull().sum().sum())

Tổng số giá trị thiếu: 0
Tổng số giá trị thiếu: 0


In [9]:
all_zero_cols = [col for col in data_binary.columns if (data_binary[col].nunique() == 1 and data_binary[col].iloc[0] == 0)]

print("Các cột toàn 0:")
for col in all_zero_cols:
    print(f"- {col}")


Các cột toàn 0:
- Bwd PSH Flags
- Bwd URG Flags
- Fwd Avg Bytes/Bulk
- Fwd Avg Packets/Bulk
- Fwd Avg Bulk Rate
- Bwd Avg Bytes/Bulk
- Bwd Avg Packets/Bulk
- Bwd Avg Bulk Rate


In [10]:
data_binary = data_binary.drop(columns=all_zero_cols)
data_multiclass = data_multiclass.drop(columns=all_zero_cols)

In [22]:
numeric_cols = data_binary.select_dtypes(include="number")
neg_count = (numeric_cols < 0).sum()
neg_percent = neg_count / len(data_binary) * 100
result = pd.DataFrame({
    "Số âm": neg_count,
    "Tỷ lệ %": neg_percent
})
result["Tỷ lệ %"] = result["Tỷ lệ %"].map(lambda x: f"{x:.10f}")
print(result)


                             Số âm       Tỷ lệ %
Destination Port                 0  0.0000000000
Flow Duration                    0  0.0000000000
Total Fwd Packets                0  0.0000000000
Total Backward Packets           0  0.0000000000
Total Length of Fwd Packets      0  0.0000000000
Total Length of Bwd Packets      0  0.0000000000
Fwd Packet Length Max            0  0.0000000000
Fwd Packet Length Min            0  0.0000000000
Fwd Packet Length Mean           0  0.0000000000
Fwd Packet Length Std            0  0.0000000000
Bwd Packet Length Max            0  0.0000000000
Bwd Packet Length Min            0  0.0000000000
Bwd Packet Length Mean           0  0.0000000000
Bwd Packet Length Std            0  0.0000000000
Flow Bytes/s                     0  0.0000000000
Flow Packets/s                   0  0.0000000000
Flow IAT Mean                    0  0.0000000000
Flow IAT Std                     0  0.0000000000
Flow IAT Max                     0  0.0000000000
Flow IAT Min        

In [19]:
data_binary = data_binary.drop(columns=["Init_Win_bytes_backward", "Init_Win_bytes_forward"])
data_multiclass = data_multiclass.drop(columns=["Init_Win_bytes_backward", "Init_Win_bytes_forward"])

In [21]:
# Chọn các cột số
numeric_cols = data_binary.select_dtypes(include="number")

# Giữ lại chỉ những hàng mà tất cả giá trị số đều >= 0
data_binary = data_binary[(numeric_cols >= 0).all(axis=1)]
data_multiclass = data_multiclass[(numeric_cols >= 0).all(axis=1)]


In [23]:
print(data_binary["Label"].value_counts())
print(data_multiclass["Label"].value_counts())

Label
BENIGN    2270365
ATTACK     555272
Name: count, dtype: int64
Label
BENIGN            2270365
DOS_DDOS           380505
PROBE_PORTSCAN     158930
CRED_WEB            13829
R2L_U2R_OTHER        2008
Name: count, dtype: int64


In [25]:
data_binary.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2825637 entries, 0 to 2828562
Data columns (total 69 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Destination Port             int64  
 1   Flow Duration                int64  
 2   Total Fwd Packets            int64  
 3   Total Backward Packets       int64  
 4   Total Length of Fwd Packets  int64  
 5   Total Length of Bwd Packets  int64  
 6   Fwd Packet Length Max        int64  
 7   Fwd Packet Length Min        int64  
 8   Fwd Packet Length Mean       float64
 9   Fwd Packet Length Std        float64
 10  Bwd Packet Length Max        int64  
 11  Bwd Packet Length Min        int64  
 12  Bwd Packet Length Mean       float64
 13  Bwd Packet Length Std        float64
 14  Flow Bytes/s                 float64
 15  Flow Packets/s               float64
 16  Flow IAT Mean                float64
 17  Flow IAT Std                 float64
 18  Flow IAT Max                 int64  
 19  Flow 

In [26]:
data_binary.describe().T

,count,mean,std,min,25%,50%,75%,max
Destination Port,"2,825,637.00","8,082.87","18,294.53",0.00,53.00,80.00,443.00,"65,535.00"
Flow Duration,"2,825,637.00","14,748,704.34","33,618,106.59",0.00,155.00,"31,286.00","3,073,944.00","119,999,998.00"
Total Fwd Packets,"2,825,637.00",5.73,74.35,1.00,2.00,2.00,5.00,"112,141.00"
Total Backward Packets,"2,825,637.00",5.55,102.96,0.00,1.00,2.00,4.00,"157,388.00"
Total Length of Fwd Packets,"2,825,637.00",526.25,"9,116.50",0.00,12.00,62.00,187.00,"12,900,000.00"
Total Length of Bwd Packets,"2,825,637.00","5,343.43","250,509.29",0.00,2.00,122.00,480.00,"399,000,000.00"
Fwd Packet Length Max,"2,825,637.00",207.18,716.59,0.00,6.00,37.00,80.00,"24,820.00"
Fwd Packet Length Min,"2,825,637.00",18.75,60.39,0.00,0.00,2.00,36.00,"2,325.00"
Fwd Packet Length Mean,"2,825,637.00",58.21,186.05,0.00,6.00,34.00,50.00,"5,940.86"
Fwd Packet Length Std,"2,825,637.00",68.85,281.16,0.00,0.00,0.00,26.16,"7,125.60"


In [31]:
numeric_cols = data_binary.select_dtypes(include="number")

corr_matrix = numeric_cols.corr().abs()

# Lấy phần tam giác trên (để tránh trùng lặp và loại đường chéo = 1)
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Tìm các cặp có corr > 0.95
high_corr = [
    (col, row, upper.loc[row, col])
    for col in upper.columns
    for row in upper.index
    if pd.notna(upper.loc[row, col]) and upper.loc[row, col] > 0.90
]

# Chuyển thành DataFrame để xem dễ hơn
high_corr_df = pd.DataFrame(high_corr, columns=["Feature 1", "Feature 2", "Correlation"])

print(high_corr_df.sort_values(by="Correlation", ascending=False))

                      Feature 1                    Feature 2  Correlation
35               CWE Flag Count                Fwd URG Flags         1.00
34               SYN Flag Count                Fwd PSH Flags         1.00
62          Subflow Bwd Packets       Total Backward Packets         1.00
41         Avg Fwd Segment Size       Fwd Packet Length Mean         1.00
52          Fwd Header Length.1            Fwd Header Length         1.00
54          Subflow Fwd Packets            Total Fwd Packets         1.00
43         Avg Bwd Segment Size       Bwd Packet Length Mean         1.00
70            Subflow Bwd Bytes  Total Length of Bwd Packets         1.00
60            Subflow Fwd Bytes  Total Length of Fwd Packets         1.00
8                 Fwd IAT Total                Flow Duration         1.00
12                  Fwd IAT Max                 Flow IAT Max         1.00
39          Average Packet Size           Packet Length Mean         1.00
36               ECE Flag Count       

In [32]:
drop_collums=["CWE Flag Count","SYN Flag Count","Subflow Bwd Packets","Avg Fwd Segment Size","Fwd Header Length.1",
              "Subflow Fwd Packets","Avg Bwd Segment Size","Subflow Bwd Bytes","Subflow Fwd Bytes","Fwd IAT Total",
              "Fwd IAT Max","Average Packet Size","ECE Flag Count","Bwd Header Length","Idle Min","Idle Mean","Total Backward Packets",
              "Fwd Packets/s","Fwd Header Length","Fwd Packet Length Std","act_data_pkt_fwd"]
data_binary = data_binary.drop(columns=drop_collums)
data_multiclass = data_multiclass.drop(columns=drop_collums)

In [35]:
data_binary.describe().T

,count,mean,std,min,25%,50%,75%,max
Destination Port,"2,825,637.00","8,082.87","18,294.53",0.00,53.00,80.00,443.00,"65,535.00"
Flow Duration,"2,825,637.00","14,748,704.34","33,618,106.59",0.00,155.00,"31,286.00","3,073,944.00","119,999,998.00"
Total Fwd Packets,"2,825,637.00",5.73,74.35,1.00,2.00,2.00,5.00,"112,141.00"
Total Length of Fwd Packets,"2,825,637.00",526.25,"9,116.50",0.00,12.00,62.00,187.00,"12,900,000.00"
Total Length of Bwd Packets,"2,825,637.00","5,343.43","250,509.29",0.00,2.00,122.00,480.00,"399,000,000.00"
Fwd Packet Length Max,"2,825,637.00",207.18,716.59,0.00,6.00,37.00,80.00,"24,820.00"
Fwd Packet Length Min,"2,825,637.00",18.75,60.39,0.00,0.00,2.00,36.00,"2,325.00"
Fwd Packet Length Mean,"2,825,637.00",58.21,186.05,0.00,6.00,34.00,50.00,"5,940.86"
Bwd Packet Length Max,"2,825,637.00",868.99,"1,945.00",0.00,2.00,79.00,276.00,"19,530.00"
Bwd Packet Length Min,"2,825,637.00",41.12,68.90,0.00,0.00,0.00,77.00,"2,896.00"


### embedding
- Destination Port
### log + scale:
- Total Fwd Packets (1 → 112,141)
- Total Length of Fwd Packets (0 → 12,900,000)
- Total Length of Bwd Packets (0 → 399,000,000)
- Fwd Packet Length Max (0 → 24,820)
- Fwd Packet Length Min (0 → 2,325)
- Fwd Packet Length Mean (0 → 5,940)
- Bwd Packet Length Max (0 → 19,530)
- Bwd Packet Length Min (0 → 2,896)
- Bwd Packet Length Mean (0 → 5,800)
- Bwd Packet Length Std (0 → 8,194)
- Flow Bytes/s (0 → 2,071,000,000)
- Flow Packets/s (0 → 4,000,000)
- Flow IAT Mean (0 → 120,000,000)
- Flow IAT Std (0 → 84,800,261)
- Flow IAT Max (0 → 120,000,000)
- Flow IAT Min (0 → 120,000,000)
- Fwd IAT Mean/Std/Min (0 → 120,000,000)
- Bwd IAT Total/Mean/Std/Max/Min (0 → 120,000,000)
- Active Mean/Std/Max/Min (0 → 110,000,000)
- Idle Max/Min/Std (0 → 120,000,000)
- Down/Up Ratio (0 → 156)
### only scale
- min_seg_size_forward(26 ± 6)
- Packet Length Std/Variance
### don't need scale
- Fwd PSH Flags, Fwd URG Flags, FIN Flag Count, RST Flag Count, PSH Flag Count, ACK Flag Count, URG Flag Count

In [33]:
data_binary.to_csv(r'.\TrafficLabelling\data_binary.csv', index=False)
data_multiclass.to_csv(r'.\TrafficLabelling\data_multi_class.csv', index=False)